In [1]:
import os
import glob
import numpy as np
import pandas as pd

from pprint import pprint

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, layers, Sequential, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print("GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
train_path = "/content/drive/MyDrive/data/disease_leaf/train"
valid_path = "/content/drive/MyDrive/data/disease_leaf/val"

In [5]:
training_images = []

for root, dirs, files in os.walk(train_path):
    for file in files:
        training_images.append(os.path.join(root, file))
                               
validation_images = []

for root, dirs, files in os.walk(valid_path):
    for file in files:
        validation_images.append(os.path.join(root, file))

In [6]:
print("Training:")
print("Training Path:" + train_path)
print("Training Classes:" + str(len(os.listdir(train_path))))
print("Training Images:" + str(len(training_images)))

print("\n")

print("Validation:")
print("Validation Path:" + valid_path)
print("Validation Classes:" + str(len(os.listdir(valid_path))))
print("Validation Images:" + str(len(validation_images)))

print("\n")

Training:
Training Path:/content/drive/MyDrive/data/disease_leaf/train
Training Classes:2
Training Images:3909


Validation:
Validation Path:/content/drive/MyDrive/data/disease_leaf/val
Validation Classes:2
Validation Images:975




In [7]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

In [8]:
batch_size = 32

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size = batch_size,
    class_mode = 'categorical',
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=True
)

validation_generator =  validation_datagen.flow_from_directory(
    valid_path,
    batch_size  = batch_size,
    class_mode  = 'categorical',
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=True
)

Found 3909 images belonging to 2 classes.
Found 975 images belonging to 2 classes.


In [10]:
class_dict = train_generator.class_indices
pprint(class_dict)

{'Pepper,_bell___Bacterial_spot': 0, 'Pepper,_bell___healthy': 1}


In [11]:
class_list = list(class_dict.keys())
pprint(class_list)

['Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy']


In [12]:
train_num = train_generator.samples
valid_num = validation_generator.samples

In [13]:
# Creating Instence of pre-trained model from Keras Application
inception_model = InceptionV3(input_shape= (150, 150, 3),
                                include_top = False,
                                weights = 'imagenet')

In [14]:
# Freezing all Layers of Inception V3 Model
for layer in inception_model.layers:
    layer.trainable = False

In [15]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [16]:
# Taking output from 'mixed8' layer
last_layer = inception_model.get_layer('mixed9')
print('Last Layer Output Shape:', last_layer.output_shape)
last_output = last_layer.output

Last Layer Output Shape: (None, 3, 3, 2048)


In [17]:
# Adding our own layers at the end of Inception Network

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(2048, activation='relu')(x)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(len(class_dict), activation='softmax')(x)

model = Model(inception_model.input, x)

In [18]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
# # checkpoint
# weightpath = "best_weights.hdf5"
# checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
# callbacks_list = [checkpoint]

In [20]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_num//batch_size,
    validation_data = validation_generator,
    validation_steps=valid_num//batch_size,
    epochs = 30,
    callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)],
    verbose = 1
)

Epoch 1/30
122/122 [==============================] - 1124s 9s/step - loss: 0.2232 - accuracy: 0.9113 - val_loss: 0.0980 - val_accuracy: 0.9604


In [21]:
model.evaluate_generator(
    generator=validation_generator,
    steps=valid_num//batch_size
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.09537424892187119, 0.9614583253860474]

In [90]:
# model.save("/content/drive/MyDrive/model.h5")

In [91]:
# model.save("/content/drive/MyDrive/mymodel.h5py")

In [ ]:
# from zipfile import ZipFile 
# import os 

# def get_all_file_paths(directory): 
  
#     # initializing empty file paths list 
#     file_paths = [] 
  
#     # crawling through directory and subdirectories 
#     for root, directories, files in os.walk(directory): 
#         for filename in files: 
#             # join the two strings in order to form the full filepath. 
#             filepath = os.path.join(root, filename) 
#             file_paths.append(filepath) 
  
#     # returning all file paths 
#     return file_paths   

In [ ]:
# # path to folder which needs to be zipped 
# directory = 'saved_models/'

# # calling function to get all file paths in the directory 
# file_paths = get_all_file_paths(directory) 

# # printing the list of all files to be zipped 
# print('Following files will be zipped:') 
# for file_name in file_paths: 
#     print(file_name) 

# # writing files to a zipfile 
# with ZipFile('saved_models.zip','w') as zip: 
#     # writing each file one by one 
#     for file in file_paths: 
#         zip.write(file) 

# print('All files zipped successfully!')       

In [9]:
import pickle
import tensorflow
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
model=keras.models.load_model("/content/drive/MyDrive/mymodel.h5py")

In [36]:
import os
import numpy as np

In [69]:
test_path = "/content/drive/MyDrive/data/disease_leaf/test"

In [70]:
test_images = []

for root, dirs, files in os.walk(test_path):
    for file in files:
        test_images.append(os.path.join(root, file))

In [73]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [75]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size  = 1,
    class_mode  = None,
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=True
)

Found 1 images belonging to 1 classes.


In [76]:
test_generator.reset()
pred=model.predict_generator(
    test_generator,
    steps=test_generator.n//test_generator.batch_size,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


1/1 [==============================] - 0s 31ms/step


In [87]:
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices

array([0])

In [88]:
print(predicted_class_indices)

[0]


In [85]:
if (predicted_class_indices==[1]):
  result='Healthy'
else:
  result='Infected'


In [89]:
result

'Infected'